## Setup & Import Data

In [1]:
# setup doc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# import data
weights = pd.read_csv("Weight.csv")

# filter out duplicate date columns
cols = ["Start","Weight (lb)"] 
weights = weights[cols] 

# rename columns
weights.columns = ['Date', 'Weight']

# convert date string to datetime object
weights['Date'] = pd.to_datetime(weights['Date'])

# copy first weight to trend column
weights.at[0,'Trend'] = weights['Weight'].iloc[0]

# Calculate the Trend given the smoothing constant c
c = 0.9
for i, row in weights[1:].iterrows():
        res = (1 - c) * (weights['Weight'][i] - weights['Trend'][i - 1])
        res = round(res,1)
        weights.at[i,'Trend'] = res + weights['Trend'][i - 1]


# copy put zeros in rate col
weights.at[0,'Lbs_per_day'] = 0

#get daily rates
for i, row in weights[1:].iterrows():
        weights.at[i,'Lbs_per_day'] = weights['Trend'][i] - weights['Trend'][i - 1]

In [2]:
weights;

## Plotting

In [3]:
# plot
startDate = pd.to_datetime("January 15, 2020")
endDate = pd.Timestamp.now()
domain = (startDate,endDate)
#weights.plot.line(figsize=(10,6),x = "Date", legend=True, xlim=domain)

## Params

In [4]:
dietStart = "January 15, 2020"
start = "April 20, 2020"
end = "April 28, 2020"

## Trend Calculations

In [5]:
def makeLinearFit(x,y):
    return np.poly1d(np.polyfit(x,y,y))

In [6]:
dietStartDate = pd.to_datetime(dietStart)

# Get subset of all weights based on date range
rangeStart = pd.to_datetime(start)
rangeEnd = pd.to_datetime(end)
dietLength = (rangeEnd - dietStartDate).days

# Get dates in range
mask = (weights['Date'] > rangeStart) & (weights['Date'] <= rangeEnd)
filteredWeights = weights[mask]

# Create linear fit
y = filteredWeights['Trend'].tolist()
x = range(len(y))
#fit = np.polyfit(x,y,1)
#fit_fn = np.poly1d(fit)
fit_fn = makeLinearFit(x,y)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

## Plots

In [ ]:
plt.figure(figsize = [15,4])
plt.plot(filteredWeights['Date'], fit_fn(x), 'k-')
plt.plot(filteredWeights['Date'], filteredWeights['Trend'], 'go', ms = 4)

## Calculate success date

In [ ]:
i = dietLength
while fit_fn(i) > 225:
    i += 1
    
fit_fn(i)
#print("Day:",i," Value: ",fit_fn(i))
goalDay = dietStartDate + pd.DateOffset(i)
print("Goal Reached: ",goalDay.month,"/",goalDay.day,"/",goalDay.year, ": ",i-dietLength, "Days from now.")
#print(np.poly1d(fit_fn))

In [ ]:
# get date range
# make fit line
# calculate date reached